In [11]:
pip install geopandas

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 323 kB 318 kB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 167 kB/s eta 0:00:01
     |████████████████████████████████| 15.1 MB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 6.6 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
# loading packages
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd 
import matplotlib.colors as mcolors
from datetime import datetime
import re

In [5]:
mpl.style.use('fivethirtyeight')

In [14]:
df=pd.read_csv('spacex_launch_data.csv.xls')
df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Mission Outcome,Landing Outcome
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,NaN,LEO,SpaceX,Success,Failure (parachute)
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",NaN,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525,LEO (ISS),NASA (COTS),Success,No attempt
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [18]:
# Rename columns
df.rename(columns={df.columns[2]: 'Time', 
                   df.columns[5]: 'Payload.Name', 
                   df.columns[6]: 'Payload.Mass'}, inplace=True)

# Convert 'Date' column to datetime and extract 'Year'
df['DateTS'] = pd.to_datetime(df['Date'])
df['Year'] = df['DateTS'].dt.year

# Clean 'Landing.Outcome' column
df['Landing.Outcome.v2'] = df['Landing Outcome'].str.replace("   ", " ").str.replace("  ", " ")

# Clean and extract 'Launch.Site.v2' before '('
df['Launch.Site.v2'] = df['Launch Site'].apply(lambda x: x.split("(")[0].strip())

# Function to determine Landing Outcome
def make_landing_outcome(x):
    y = x.split(" ")[0]  # Extract first word
    return 'No Attempt' if y == 'No' else y

df['Landing.Outcome.v3'] = df['Landing.Outcome.v2'].apply(make_landing_outcome)

# Clean and convert 'Payload.Mass' to numeric
df['Payload.Mass'] = df['Payload.Mass'].astype(str).str.strip()
df['Payload.Mass'] = df['Payload.Mass'].str.replace(",", "").str.replace("~", "")
df['Payload.Mass'] = pd.to_numeric(df['Payload.Mass'], errors='coerce')  # Convert to numeric, handling errors

# Display the DataFrame
df.head()

,Flight Number,Date,Time,Booster Version,Launch Site,Payload.Name,Payload.Mass,Orbit,Customer,Mission Outcome,Landing Outcome,DateTS,Year,Landing.Outcome.v2,Launch.Site.v2,Landing.Outcome.v3
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,NaN,LEO,SpaceX,Success,Failure (parachute),2010-06-04,2010,Failure (parachute),CCAFS LC-40,Failure
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",NaN,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute),2010-12-08,2010,Failure (parachute),CCAFS LC-40,Failure
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),Success,No attempt,2012-05-22,2012,No attempt,CCAFS LC-40,No Attempt
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),Success,No attempt,2012-10-08,2012,No attempt,CCAFS LC-40,No Attempt
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),Success,No attempt,2013-03-01,2013,No attempt,CCAFS LC-40,No Attempt
